# AEFIT

This would be the first attempt to run the unsupervised learning VAE network to learn how to characterize a 1D profile with atted noise and missing input.

More than a simple fit, this method should learn 

In [2]:
import numpy as np
import tensorflow as tf

# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors 

import ipysh
import Hunch_utils  as Htls
import Hunch_lsplot as Hplt

%aimport Dummy_g1data
import Dummy_g1data as dummy
%aimport models.base
%aimport models.AEFIT
%aimport models.AEFIT3

# ipysh.Bootstrap_support.debug()

## Data and Model
The model and data generator are set:
Dummy data generator generates from a set of 5 kind of curves with a dataset cardinality of 60K samples.

All the shapes are generated from a dictionary array that defines mean sigma and gain of sum of gaussians.
This table is printed from the variable ds.kinds

>NOTE: 
> The actual model is generating random so it is not redoing the very same samples on each epoch.
> To exactly constraint the maximum size of the dataset the tf buffer can be used

the model uses bby default an input of 40 samples that are the (x,y) tuple values of 20 points from the generated shapes.


In [3]:
ds = dummy.Dummy_g1data(counts=40000).buffer()
ds.kinds

[{'gain': [1, 1], 'mean': [0.2, 0.8], 'sigma': [0.1, 0.1]},
 {'gain': [0.5], 'mean': [0.8], 'sigma': [0.1]},
 {'gain': [0.5], 'mean': [0.2], 'sigma': [0.1]},
 {'gain': [1], 'mean': [0.5], 'sigma': [0.2]},
 {'gain': [0.5], 'mean': [0.5], 'sigma': [0.2]}]

In [ ]:
# m = aefit.AEFIT(latent_dim=2)
# aefit.test_dummy(m, data=ds, epoch=5, batch=200, loss_factor=1e-3)

In [ ]:
dds = ds.ds_array.map(lambda xy,l: (xy,l) )
# [x for x in dds.take(1)][0]

In [4]:
m3 = models.AEFIT3.AEFIT3(latent_dim=2, beta=30.)

AEFIT3 ready:


In [5]:
models.base.train_thread(m3, ds, epoch=5, batch=200, learning_rate=1e-3).control_panel()


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 1/5


W0816 14:44:29.553987 140645926684416 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


200/200 [==============================] - 17s 87ms/step - loss: 20.5690 - accuracy: 0.6555 - sce: 22.0595 - akl: 19.4784 - kld: 10.7307 - v_mea: -0.9083 - v_std: 0.4004

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0009000000427477062.
Epoch 2/5
200/200 [==============================] - 16s 79ms/step - loss: 18.5284 - accuracy: 0.6793 - sce: 18.8027 - akl: 53.2439 - kld: 27.9134 - v_mea: -2.1949 - v_std: 0.0100

Epoch 00003: LearningRateScheduler reducing learning rate to 0.000729000034625642.
Epoch 3/5
200/200 [==============================] - 13s 67ms/step - loss: 18.0412 - accuracy: 0.6797 - sce: 18.0654 - akl: 59.5188 - kld: 31.4937 - v_mea: -1.6804 - v_std: 0.0058

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0005314410154824146.
Epoch 4/5
200/200 [==============================] - 15s 74ms/step - loss: 17.9413 - accuracy: 0.6895 - sce: 17.9386 - akl: 62.0989 - kld: 33.5425 - v_mea: -1.6333 - v_std: 0.0030

Epoch 00005: LearningRateScheduler 

In [8]:
p = Hplt.LSPlotBokeh()
p.set_model(m3)
p.set_data(ds)
p.plot(notebook_url='http://172.17.0.2:8888')

Loading BokehJS ...

## Missing data simulation

Now we want to test the network against particular shapes within the latent main paths but with added noise and simulated missing data. The function simulate_missing_data reduce de number of available input simply duplicating the point that precedes the missing one with the same value.

A further gaussian noise has been also applied.

In [ ]:

def simulate_missing_data(m, lpt=[0.5,-1.6], noise_var=0.05, arr = [3,2,1,5,8,7,6,9,12,11,14,13,18]):
    xy = m.decode(tf.convert_to_tensor([pt]),apply_sigmoid=True)
    x,y = tf.split(xy[0], 2)
    x,y = (x.numpy(), y.numpy())

    fig = plt.figure('gen_missing_curve',figsize=(18, 6))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)    
    
    ax1.set_xlim(-2.,2.)
    ax1.set_ylim(-2.,2.)
    
    ax1.scatter(pt[0],pt[1],s=80)
    ax2.scatter(x,y,s=40)

    # apply noise
    x += np.random.normal(0,noise_var,len(x))
    y += np.random.normal(0,noise_var,len(y))

    # apply missing data simulation
    for i,v in enumerate(arr,0):
        x[arr[i]]=x[arr[i]+1]
        y[arr[i]]=y[arr[i]+1]
    
    ax2.scatter(x,y,s=80)

    me,va = m.encode(tf.reshape(tf.concat([x,y],0), shape=[1,-1]))
    print("Guessed Latent point = ",me.numpy())
    gpt = me[0].numpy()
    ax1.scatter(gpt[0],gpt[1])
    
    XY = m.decode(me,apply_sigmoid=True)
    X,Y = tf.split(XY[0], 2)
    X,Y = (X.numpy(), Y.numpy())
    # plt.figure('reconstructed')
    ax2.scatter(X,Y,s=40)
    # plt.plot(X,Y)



We start by generating close to the shape {'mean': [0.5], 'sigma': [0.2], 'gain': [0.5]}

This is in the middle of the central cluster

In [ ]:

    
# generate from point: 0.6, -0.7
pt = [0.6,-0.7]
noise_var = 0.1
arr = [3,2,1,5,8,7,6,9,12,11,14,13,18]
simulate_missing_data(m3, pt,noise_var,arr)


Now we want to check if the nework can simulate a point in the middle od two clusters

In [ ]:
# generate from point: 0.5, -1.6
pt = [0.5,-1.6]
noise_var = 0.05
arr = [3,2,1,5,8,7,6,9,12,11,14,13,18]
simulate_missing_data(m3, pt,noise_var,arr)

